# Random forest/XGboost prediction of C<sub>Q</sub> with DFT computed <sup>27</sup>Al tensors

DFT computed tensors was calculated to produce corresponding DFT CQ as the label. Structural and elemental features are derived from the materials' geometry optimized structural information (using VASP and customized python code in /src). The hyper parameters of the machine learning models are optimized using randomgridsearch from sklearn package. This note book is divided into 3 sections:

1. ETL of original tensor and structural information.
2. EDA and visualizations of the original data.
3. Model training and reporting.

In [ ]:
import json
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from pymatgen.core.structure import Structure as ST

# 1. ETL of original tensor and structural information.

### 1.1 Get structures and NMR raw tensors.
------------------------------------------------------------

In [ ]:
with open("data/Alnmr.json", "r") as file:
    data = json.load(file)
    print("length of file is {}".format(len(data)))

In [ ]:
# test if there's any structure dosen't contain the target atom ('Al') and does not contain a structure section
problem_compound = []
for compound in data:
    if "structure" not in compound.keys():
        problem_compound.append(compound)
        continue
    sites = []
    for site in compound["structure"]["sites"]:
        sites.append(site["label"])
    if "Al" not in sites:
        problem_compound.append(compound)
print("num of problem compound:", len(problem_compound))

for compound in problem_compound:
    data.remove(compound)
print("len of none problematic data:", len(data))

In [ ]:
# get rid of the redundances
for i in range(len(data)):
    string = json.dumps(data[i], sort_keys=True)
    data[i] = string
data = list(set(data))

for i in range(len(data)):
    dictionary = json.loads(data[i])
    data[i] = dictionary
print("length of file without redundancy is {}".format(len(data)))

In [ ]:
# get the structure_tensors obj
from src.structure_tensors_gen import get_structure_tensors

structure_tensors = get_structure_tensors(data)
print("length of structure_tensors:", len(structure_tensors))

In [ ]:
# Here's what a structure_tensors obj looks like:
structure_tensors[0]["tensors"]

### 1.2 Data cleaning and pre-training preparations 
---------------------------------------------------------------

In [ ]:
from src.structure_tensors_modifier import (
    get_n_coord_tensors,
    append_coord_num,
    add_oxi_state_by_guess,
)

# Add oxidation state for each structures in structure_tensors obj. Might take a long time based on the structure.
structure_tensors = add_oxi_state_by_guess(structure_tensors)
# Filter structure based on coordination number and append coord num info to "tensors".
structure_tensors_filtered = get_n_coord_tensors(structure_tensors, coord=[4, 5, 6])
structure_tensors_filtered = append_coord_num(structure_tensors_filtered)
len(structure_tensors_filtered)

In [ ]:
# Add chemical environment info to "Tensor" list. Might take a long time based on the structure.
from src.structure_tensors_modifier import append_ce

structure_tensors_filtered = append_ce(structure_tensors_filtered)

In [ ]:
# Filter structures based on local chemenv. Here we select T:4 T:5 O:6 sites
from src.structure_tensors_modifier import filter_ce

chemenv_filter = filter_ce(structure_tensors_filtered)
# number of outliers
print("number of outliers:", len(chemenv_filter["outliers"]))

In [ ]:
# Save the processed data for later direct use
processed_data = copy.deepcopy(chemenv_filter["filtered"])
for data in processed_data:
    data["structure"] = data["structure"].as_dict()
dir_ = "./data/"
filename = "processed_data_0.5.json"
with open(dir_ + filename, "w") as outfile:
    json.dump(processed_data, outfile)

# 2. EDA and visualizations of the original data.

### 2.1 Feature generation

In [ ]:
# Read processed data and continue
with open("./data/processed_data_0.5.json", "r") as file:
    data_reload = json.load(file)
for data in data_reload:
    data["structure"] = ST.from_dict(data["structure"])
print("number of structures:", len(data_reload))

In [ ]:
# Calculate the structural and elemental features.
from src.Utility import features_gen

nmr_struc_data = features_gen(data_reload)
nmr_struc_data.reset_index(drop=True, inplace=True)
nmr_struc_data.head()

In [ ]:
# The dataset is not balanced in terms of neighbor atoms' types, so we need to determine
# the proportion of Al sites that have pure oxygen neighbors or not.
def is_O(combo):
    if combo == "O":
        return True
    else:
        return False


nmr_struc_data.insert(
    loc=0, column="is_O", value=nmr_struc_data["atom_combination"].map(is_O)
)

In [ ]:
# percentage of is_O sites
percent = len(nmr_struc_data[nmr_struc_data["is_O"] == True]) / len(
    nmr_struc_data["is_O"]
)
print(f"percentage of is_O sites:{percent}\n", f"not is_O sites:{1-percent}")

### 2.2  Visualizations

In [ ]:
# plot the distribution of CQ wrt the types local geometry.
plt.figure(figsize=(10, 6))
sns.histplot(data=nmr_struc_data, x="CQ", hue="max_ce")
plt.show()

In [ ]:
# plot thr distribution of CQ wrt to is_O.
plt.figure(figsize=(10, 6))
sns.histplot(data=nmr_struc_data, x="CQ", hue="is_O")
plt.show()

In [ ]:
# heat map of structural features
feature_rank = [
    "CQ",
    "fbl_std",
    "DI",
    "fba_std",
    "fba_max",
    "fba_average",
    "fbl_max",
    "fbl_average",
    "fba_min",
    "fbl_min",
]
heatmap_data = nmr_struc_data.loc[:, "CQ":"DI"][feature_rank]

# rename features for easier understanding
feature_rename = {
    "fbl_std": "std(fbl)",
    "fbl_min": "min(fbl)",
    "fba_std": "std(fba)",
    "fba_max": "max(fba)",
    "fba_average": "mean(fba)",
    "fbl_max": "max(fbl)",
    "fbl_average": "mean(fbl)",
    "fba_min": "min(fba)",
}
heatmap_data.rename(columns=feature_rename, inplace=True)

corr = heatmap_data.corr()

sns.set(font_scale=1.3)
plt.figure(figsize=[15, 12])
heat_map = sns.heatmap(
    corr,
    vmin=-1,
    vmax=1,
    center=0,
    cmap=sns.diverging_palette(220, 220, n=200),
    # cmap = sns.color_palette("ch:start=.2,rot=-.3", as_cmap=True),
    square=True,
    annot=True,
    annot_kws={"size": 14},
)
heat_map.set_xticklabels(
    heat_map.get_xticklabels(), rotation=45, horizontalalignment="right"
)
heat_map.set_yticklabels(
    heat_map.get_yticklabels(), rotation=0, horizontalalignment="right"
)

plt.savefig("./figures/27Al_color_map.png", format="png", bbox_inches="tight")
plt.show()

In [ ]:
# replot without high correlation features: fba_std,fba_max,fba_min,fbl_max,fbl_min

try:
    corr = corr.drop(
        ["std(fba)", "max(fba)", "min(fba)", "max(fbl)", "min(fbl)"], axis=0
    )
    corr = corr.drop(
        ["std(fba)", "max(fba)", "min(fba)", "max(fbl)", "min(fbl)"], axis=1
    )
except KeyError:
    pass

sns.set(font_scale=1.3)
plt.figure(figsize=[10, 8])
heat_map = sns.heatmap(
    corr,
    vmin=-1,
    vmax=1,
    center=0,
    cmap=sns.diverging_palette(220, 220, n=200),
    square=True,
    annot=True,
    annot_kws={"size": 14},
)
heat_map.set_xticklabels(
    heat_map.get_xticklabels(), rotation=45, horizontalalignment="right"
)
heat_map.set_yticklabels(
    heat_map.get_yticklabels(), rotation=0, horizontalalignment="right"
)

plt.show()

In [ ]:
# Noticed that the distrotion index can not correctly represent some of the local geometries. (red labeled ones)
red_labels = nmr_struc_data[(nmr_struc_data["DI"] < 0.001) & (nmr_struc_data["CQ"] > 0)]

fig, ax = plt.subplots(figsize=[10, 6])
sns.scatterplot(data=nmr_struc_data, x="DI", y="CQ", ax=ax)
sns.scatterplot(data=red_labels, x="DI", y="CQ", ax=ax, color="red")
plt.show()

In [ ]:
# And these unrepresented geometries are nicely represented by the standard deviation of first order bond length.
fig, ax = plt.subplots(figsize=[10, 6])
sns.scatterplot(data=nmr_struc_data, x="fbl_std", y="CQ", ax=ax)
sns.scatterplot(data=red_labels, x="fbl_std", y="CQ", ax=ax, color="red")
plt.show()

# 3. Model training and reporting
------------------------------------------------
1. Random forest
2. GBDT (XGboost)

### 3.1 Train test split and rebalance

In [ ]:
# split y and x
y = nmr_struc_data[["CQ", "is_O"]]
x = nmr_struc_data.loc[:, "fbl_average":]
# x = data_nocollinear

# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

print(f"Size of train set: {len(X_train)}\nSize of test set: {len(X_test)}")

In [ ]:
# resample the dataset using SMOTE to make ti balance
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

train = pd.concat([X_train, y_train["CQ"]], axis=1)
label = y_train["is_O"]

over = SMOTE(sampling_strategy=0.75)
under = RandomUnderSampler(sampling_strategy=1.0)
steps = [("o", over), ("u", under)]
pipeline = Pipeline(steps=steps)

train, label = pipeline.fit_resample(train, label)
y_train = pd.concat([train["CQ"], label], axis=1)
X_train = train.drop(columns=["CQ"])

In [ ]:
# now the data is balanced
plt.figure(figsize=(10, 6))
sns.histplot(data=pd.concat([X_train, y_train], axis=1), x="CQ", hue="is_O")
plt.show()

### 3.2 Random forest

3.2.1. Model Training with RandomSearchCV

In [ ]:
%%time
# Random Search for Algorithm Tuning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math

# create and fit a kernel ridge regression model
model = RandomForestRegressor(random_state=10)

param = {
    "n_estimators": randint(low=10, high=1000),
    "max_depth": randint(low=10, high=50),
    "min_samples_split": randint(low=2, high=10),
    "min_samples_leaf": randint(low=1, high=8),
    "max_features": [None, "sqrt", "log2"],
}

grid_rf = RandomizedSearchCV(
    estimator=model,
    param_distributions=param,
    n_iter=10,
    scoring=["neg_mean_absolute_error", "neg_mean_squared_error", "r2"],
    refit="r2",
    cv=10,
    n_jobs=8,
)
grid_rf.fit(X_train, y_train["CQ"])

# summarize the results of the grid search
train_r2_mean = np.sort(grid_rf.cv_results_["mean_test_r2"])[-1]
train_RMSE_mean = math.sqrt(
    -np.sort(grid_rf.cv_results_["mean_test_neg_mean_squared_error"])[-1]
)
train_MAE_mean = -np.sort(grid_rf.cv_results_["mean_test_neg_mean_absolute_error"])[-1]

print(
    "training score: R2_mean = {}, RMSE_mean = {}, MAE_mean = {}".format(
        train_r2_mean, train_RMSE_mean, train_MAE_mean
    )
)
print(grid_rf.best_estimator_)

In [ ]:
grid_rf.best_estimator_.get_params()

In [ ]:
# plot feature importance
feat_importances = pd.Series(
    grid_rf.best_estimator_.feature_importances_, index=X_train.columns
)
feat_importances.nlargest(10).plot(kind="barh")
plt.show()

In [ ]:
# print training score
y_train_predict = grid_rf.predict(X_train)

train_r2 = r2_score(y_train["CQ"], y_train_predict)
train_RMSE = math.sqrt(mean_squared_error(y_train["CQ"], y_train_predict))
train_MAE = mean_absolute_error(y_train["CQ"], y_train_predict)

print(
    "Train scores: R2 = {}, RMSE = {}, MAE = {}".format(train_r2, train_RMSE, train_MAE)
)

In [ ]:
# Predict test set
from datetime import datetime
from src.Utility import reg_plot


def print_test_results(table):
    test_r2 = r2_score(table["VASP_CQ"], table["RF_CQ"])
    test_RMSE = math.sqrt(mean_squared_error(table["VASP_CQ"], table["RF_CQ"]))
    test_MAE = mean_absolute_error(table["VASP_CQ"], table["RF_CQ"])
    print(
        "test scores: R2 = {}, RMSE = {}, MAE = {}".format(test_r2, test_RMSE, test_MAE)
    )


y_rf = pd.Series(grid_rf.predict(X_test))
y_test.reset_index(drop=True, inplace=True)

test_result = pd.concat([y_test, y_rf], axis=1)
test_result.rename(columns={"CQ": "VASP_CQ", 0: "RF_CQ"}, inplace=True)

print_test_results(test_result)


# write down the date for png save
predict_result = {}
predict_result["VASP_CQ"] = y_test["CQ"]
predict_result["RF_predicted_CQ"] = y_rf
predict_result = pd.DataFrame(predict_result)

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")
print("date and time:", dt_string)

# plot the correlation
reg_plot(
    test_result["VASP_CQ"],
    test_result["RF_CQ"],
    "VASP calculated CQ (MHz)",
    "Random Forest predicted CQ (MHz)",
)

# # Export y_rf and y_test as .csv
# y_output = copy.deepcopy(y_test)
# y_output['CQ_rf'] = y_rf
# y_output = pd.DataFrame(y_output)
# y_output.to_csv('./data/All_feature_test.csv')

In [ ]:
# compare the result between is_O sites and not is_O sites
test_result_O = test_result[test_result["is_O"] == True]
test_result_notO = test_result[test_result["is_O"] == False]

print_test_results(test_result_O)
print_test_results(test_result_notO)

sns.set(font_scale=1.5)
plot = sns.lmplot(
    x="VASP_CQ", y="RF_CQ", data=test_result, hue="is_O", height=6, aspect=5 / 4
)
plot.set(xlabel="VASP calculated CQ (MHz)", ylabel="Random Forest predicted CQ (MHz)")
plt.show()

3.2.2 Learning curves

Learning curves with respect to max_depth and max_features

In [ ]:
# Get the learning curve
from src.Utility import learning_curve_param
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    random_state=10,
    max_depth=50,
    n_estimators=500,
    min_samples_split=4,
    min_samples_leaf=2,
)
param_name = "max_features"
# feature_values = np.array(range(1,21))*(X_train.shape[1]//20)
param_values = [1, 2, 3, 4, 5, 6, 7, 8, 9]
learning_curve_dict = learning_curve_param(
    model, X_train, y_train["CQ"], X_test, y_test["CQ"], param_name, param_values
)
pd.DataFrame(learning_curve_dict)

In [ ]:
pd.DataFrame(learning_curve_dict).to_csv(
    "./data/learning_curve_max_features_structure_all.csv"
)

Learning curves wrt sample size

In [ ]:
# get a series of smaller datasets randomly selected with size 10% - 100% of the total dataset
small_sets = []
for p in range(1, 11):
    small_sets.append(nmr_struc_data.sample(frac=p / 10, random_state=20))

In [ ]:
from src.Utility import learning_curve_samplesize

model = RandomForestRegressor(
    random_state=10,
    min_samples_split=4,
    min_samples_leaf=2,
    max_depth=50,
    n_estimators=500,
    max_features="sqrt",
)
feature_names = small_sets[0].loc[:, "fbl_average":].columns
learning_curve_dict = learning_curve_samplesize(model, small_sets, feature_names)
pd.DataFrame(learning_curve_dict)

In [ ]:
pd.DataFrame(learning_curve_dict).to_csv(
    "./data/learning_curve_samplesize_structure_and_elemental.csv"
)

### 3.3 GBDT (XGboost)

In [ ]:
import xgboost
from sklearn.model_selection import RandomizedSearchCV
import math

model = xgboost.XGBRegressor(tree_method="hist")

param = {
    "learning_rate": uniform(0, 1),
    "max_depth": randint(3, 50),
    "min_child_weight": randint(1, 10),
    "eta": uniform(0.01, 0.2),
    "gamma": uniform(0, 1),
    "reg_alpha": [1e-5, 1e-2, 0.1, 1, 100],
    "subsample": uniform(0, 1),
    "colsample_bytree": uniform(0, 1),
}
grid_gb = RandomizedSearchCV(
    estimator=model,
    param_distributions=param,
    n_iter=10,
    scoring=["neg_mean_absolute_error", "neg_mean_squared_error", "r2"],
    refit="r2",
    cv=5,
)
grid_gb.fit(X_train, y_train["CQ"])

# summarize the results of the grid search
train_r2 = np.sort(grid_gb.cv_results_["mean_test_r2"])[-1]
train_RMSE = math.sqrt(
    -np.sort(grid_gb.cv_results_["mean_test_neg_mean_squared_error"])[-1]
)
train_MAE = -np.sort(grid_gb.cv_results_["mean_test_neg_mean_absolute_error"])[-1]

print(
    "training score: R2 = {}, RMSE = {}, MAE = {}".format(
        train_r2, train_RMSE, train_MAE
    )
)
print(grid_gb.best_estimator_)

In [ ]:
%%time
# Predict test set
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import seaborn as sns

sns.set()
import matplotlib.pyplot as plt
from datetime import datetime

y_rf = grid_gb.predict(X_test)

test_r2 = r2_score(y_test["CQ"], y_rf)
test_RMSE = math.sqrt(mean_squared_error(y_test["CQ"], y_rf))
test_MAE = mean_absolute_error(y_test["CQ"], y_rf)

print("test scores: R2 = {}, RMSE = {}, MAE = {}".format(test_r2, test_RMSE, test_MAE))


# write down the date for png save
predict_result = {}
predict_result["VASP_CQ"] = y_test["CQ"]
predict_result["RF_predicted_CQ"] = y_rf
predict_result = pd.DataFrame(predict_result)

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")
print("date and time:", dt_string)

sns.set_style("ticks")
fig, ax = plt.subplots(figsize=(10, 8))
plot = sns.regplot(
    x="RF_predicted_CQ",
    y="VASP_CQ",
    data=predict_result,
    ci=None,
    scatter_kws={"color": "black"},
    line_kws={"color": "red"},
)
ax.set_xlabel("XGBoost predicted CQ (MHz)", fontsize=20)
ax.set_ylabel("VASP calculated CQ (MHz)", fontsize=20)
sns.despine()
# plt.savefig('./figures/27Al_RF_testSet_{}.png'.format(dt_string))
plt.show()

In [ ]:
# save the model to disk
import pickle

dir_ = "./models/best/"
filename = "Best_model_060722.sav"
pickle.dump(grid_gb, open(dir_ + filename, "wb"))